In [17]:
import pandas as pd
import requests
import json

In [18]:
url = "https://www.otomoto.pl/graphql"
raw_cars = []
wanted = ["id", "title", "createdAt", "shortDescription", "url"]
all_parameters = []

In [19]:
for page in range(1, 501):
    querystring = {
        "operationName":"listingScreen",
        "variables":"{\"categoryId\":\"29\",\"click2BuyExperimentId\":\"CARS-34184\",\"click2BuyExperimentVariant\":\"a\",\"experiments\":[{\"key\":\"MCTA-682\",\"variant\":\"a\"}],\"filters\":[{\"name\":\"category_id\",\"value\":\"29\"}],\"includeClick2Buy\":true,\"includeFiltersCounters\":false,\"includePriceEvaluation\":true,\"includePromotedAds\":false,\"includeRatings\":false,\"includeSortOptions\":false,\"maxAge\":60,\"page\":"+str(page)+",\"parameters\":[\"offer_type\",\"make\",\"model\",\"version\",\"generation\",\"year\",\"mileage\",\"engine_capacity\",\"fuel_type\",\"engine_power\",\"color\", \"colour_type\",\"gearbox\",\"transmission\", \"consumption\", \"urban_consumption\", \"body_type\", \"door_count\", \"nr_seats\", \"country_origin\",\"air_conditioning_type\", \"user_status\", \"upholstery_type\", \"cruisecontrol_type\", \"alloy_wheels_type\"],\"searchTerms\":null,\"sortBy\":null}","extensions":"{\"persistedQuery\":{\"sha256Hash\":\"4296766cd60f6b58945af803b580d7d849ed23caa88454ec8e22e5f53a31812e\",\"version\":1}}"
    }
    response = requests.request("GET", url, params=querystring)
    raw_cars += json.loads(response.text)["data"]["advertSearch"]["edges"]

In [25]:
for car in raw_cars:
    parameters = {k: car["node"][k] for k in wanted}
    parameters.update({k: car["node"]["price"]["amount"][k] for k in ["units", "currencyCode"]})
    parameters.update({k: car["node"]["location"][k]["name"] for k in ["city", "region"]})
    parameters.update({el["key"]: el["value"] for el in car["node"]["parameters"]})
    parameters.update({"seller_id": car["node"]["sellerLink"]["id"]})
    all_parameters.append(parameters)

In [26]:
df = pd.DataFrame(all_parameters)
df.info()

In [30]:
df = df.drop_duplicates()
df.to_csv("cars.csv", sep=";", index=False)